## Session 15 Assignment 1

In this assignment, students will be using the K-nearest neighbors algorithm to predict how many points NBA players scored in the 2013-2014 season.

A look at the data

Before we dive into the algorithm, let’s take a look at our data. Each row in the data contains information on how a player performed in the 2013-2014 NBA season.

Download 'nba_2013.csv' file from this link:

https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0

Here are some selected columns from the data:

player - name of the player

pos - the position of the player

g - number of games the player was in

gs - number of games the player started

pts - total points the player scored

There are many more columns in the data, mostly containing information about average player game performance over the course of the season. See this site for an explanation of the rest of them.

We can read our dataset in and figure out which columns are present:

In [13]:
# Importing module and assigning alias for them.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
# Creating "nba" data-frame with CSV file.
with open("nba_2013.csv", 'r') as csvfile:
    nba = pd.read_csv(csvfile)
nba.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [15]:
numeric_col = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
nba[numeric_col]

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,23,63,0,847,66,141,0.468,4,15,0.266667,...,0.660,72,144,216,28,23,26,30,122,171
1,20,81,20,1197,93,185,0.503,0,0,NaN,...,0.581,142,190,332,43,40,57,71,203,265
2,27,53,12,961,143,275,0.520,0,0,NaN,...,0.639,102,204,306,38,24,36,39,108,362
3,28,73,73,2552,464,1011,0.459,128,300,0.426667,...,0.815,32,230,262,248,35,3,146,136,1330
4,25,56,30,951,136,249,0.546,0,1,0.000000,...,0.836,94,183,277,40,23,46,63,187,328
5,25,46,2,330,33,61,0.541,0,0,NaN,...,0.867,37,92,129,14,8,30,18,40,92
6,28,69,69,2498,652,1423,0.458,3,15,0.200000,...,0.822,166,599,765,178,63,68,123,147,1603
7,24,65,2,1072,134,300,0.447,2,13,0.153846,...,0.660,119,192,311,71,24,33,44,126,303
8,38,73,9,1936,240,543,0.442,116,309,0.375405,...,0.905,23,182,205,143,54,8,84,115,701
9,32,55,28,1278,204,413,0.494,11,47,0.234043,...,0.628,79,129,208,94,90,19,90,121,495


In [16]:
# Creating data frame with numeric colums
nba_numeric = nba[numeric_col]
nba_numeric

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,23,63,0,847,66,141,0.468,4,15,0.266667,...,0.660,72,144,216,28,23,26,30,122,171
1,20,81,20,1197,93,185,0.503,0,0,NaN,...,0.581,142,190,332,43,40,57,71,203,265
2,27,53,12,961,143,275,0.520,0,0,NaN,...,0.639,102,204,306,38,24,36,39,108,362
3,28,73,73,2552,464,1011,0.459,128,300,0.426667,...,0.815,32,230,262,248,35,3,146,136,1330
4,25,56,30,951,136,249,0.546,0,1,0.000000,...,0.836,94,183,277,40,23,46,63,187,328
5,25,46,2,330,33,61,0.541,0,0,NaN,...,0.867,37,92,129,14,8,30,18,40,92
6,28,69,69,2498,652,1423,0.458,3,15,0.200000,...,0.822,166,599,765,178,63,68,123,147,1603
7,24,65,2,1072,134,300,0.447,2,13,0.153846,...,0.660,119,192,311,71,24,33,44,126,303
8,38,73,9,1936,240,543,0.442,116,309,0.375405,...,0.905,23,182,205,143,54,8,84,115,701
9,32,55,28,1278,204,413,0.494,11,47,0.234043,...,0.628,79,129,208,94,90,19,90,121,495


In [17]:
# Normalizing the numeric data frame
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()
nba_normalized

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,-0.835906,0.384886,-0.862207,-0.435088,-0.738401,-0.768505,0.319884,-0.700282,-0.716608,-0.117009,...,-0.389712,0.260690,-0.129462,-0.013116,-0.645220,-0.468056,0.061410,-0.667650,0.226515,-0.734621
1,-1.550487,1.095711,-0.187863,-0.045011,-0.581271,-0.649215,0.674593,-0.778936,-0.829601,NaN,...,-0.882950,1.387883,0.187020,0.565852,-0.530733,0.020680,1.065446,-0.013760,1.363938,-0.534801
2,0.116868,-0.010016,-0.457600,-0.308035,-0.290291,-0.405214,0.846880,-0.778936,-0.829601,NaN,...,-0.520826,0.743773,0.283340,0.436083,-0.568895,-0.439307,0.385292,-0.524113,0.029924,-0.328603
3,0.355062,0.779789,1.599148,1.465144,1.577804,1.590172,0.228673,1.737992,1.430256,0.898007,...,0.578033,-0.383420,0.462221,0.216475,1.033919,-0.123066,-0.683520,1.182380,0.423107,1.729123
4,-0.359519,0.108454,0.149309,-0.319180,-0.331028,-0.475703,1.110379,-0.778936,-0.822068,-1.808704,...,0.709147,0.614951,0.138859,0.291341,-0.553630,-0.468056,0.709175,-0.141348,1.139262,-0.400878
5,-0.359519,-0.286448,-0.794772,-1.011288,-0.930448,-0.985395,1.059706,-0.778936,-0.829601,NaN,...,0.902696,-0.302906,-0.487224,-0.447342,-0.752074,-0.899294,0.190963,-0.859032,-0.924950,-0.902555
6,0.355062,0.621828,1.464279,1.404961,2.671891,2.707155,0.218538,-0.719946,-0.716608,-0.539933,...,0.621738,1.774349,3.000954,2.727000,0.499648,0.681912,1.421717,0.815564,0.577572,2.309453
7,-0.597712,0.463867,-0.794772,-0.184324,-0.342667,-0.337436,0.107058,-0.739609,-0.731674,-0.832726,...,-0.389712,1.017520,0.200780,0.461039,-0.317024,-0.439307,0.288128,-0.444371,0.282684,-0.454022
8,2.736998,0.779789,-0.558752,0.778608,0.274211,0.321367,0.056385,1.502030,1.498052,0.572808,...,1.139950,-0.528344,0.131979,-0.068018,0.232512,0.423169,-0.521579,0.193571,0.128220,0.392026
9,1.307836,0.068964,0.081875,0.045264,0.064705,-0.031079,0.583382,-0.562638,-0.475557,-0.323972,...,-0.589504,0.373410,-0.232662,-0.053045,-0.141478,1.458140,-0.165308,0.289262,0.212473,-0.045878


In [18]:
# Filling "NAN" value with zero
nba_normalized.fillna(0, inplace=True)
nba_normalized

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,-0.835906,0.384886,-0.862207,-0.435088,-0.738401,-0.768505,0.319884,-0.700282,-0.716608,-0.117009,...,-0.389712,0.260690,-0.129462,-0.013116,-0.645220,-0.468056,0.061410,-0.667650,0.226515,-0.734621
1,-1.550487,1.095711,-0.187863,-0.045011,-0.581271,-0.649215,0.674593,-0.778936,-0.829601,0.000000,...,-0.882950,1.387883,0.187020,0.565852,-0.530733,0.020680,1.065446,-0.013760,1.363938,-0.534801
2,0.116868,-0.010016,-0.457600,-0.308035,-0.290291,-0.405214,0.846880,-0.778936,-0.829601,0.000000,...,-0.520826,0.743773,0.283340,0.436083,-0.568895,-0.439307,0.385292,-0.524113,0.029924,-0.328603
3,0.355062,0.779789,1.599148,1.465144,1.577804,1.590172,0.228673,1.737992,1.430256,0.898007,...,0.578033,-0.383420,0.462221,0.216475,1.033919,-0.123066,-0.683520,1.182380,0.423107,1.729123
4,-0.359519,0.108454,0.149309,-0.319180,-0.331028,-0.475703,1.110379,-0.778936,-0.822068,-1.808704,...,0.709147,0.614951,0.138859,0.291341,-0.553630,-0.468056,0.709175,-0.141348,1.139262,-0.400878
5,-0.359519,-0.286448,-0.794772,-1.011288,-0.930448,-0.985395,1.059706,-0.778936,-0.829601,0.000000,...,0.902696,-0.302906,-0.487224,-0.447342,-0.752074,-0.899294,0.190963,-0.859032,-0.924950,-0.902555
6,0.355062,0.621828,1.464279,1.404961,2.671891,2.707155,0.218538,-0.719946,-0.716608,-0.539933,...,0.621738,1.774349,3.000954,2.727000,0.499648,0.681912,1.421717,0.815564,0.577572,2.309453
7,-0.597712,0.463867,-0.794772,-0.184324,-0.342667,-0.337436,0.107058,-0.739609,-0.731674,-0.832726,...,-0.389712,1.017520,0.200780,0.461039,-0.317024,-0.439307,0.288128,-0.444371,0.282684,-0.454022
8,2.736998,0.779789,-0.558752,0.778608,0.274211,0.321367,0.056385,1.502030,1.498052,0.572808,...,1.139950,-0.528344,0.131979,-0.068018,0.232512,0.423169,-0.521579,0.193571,0.128220,0.392026
9,1.307836,0.068964,0.081875,0.045264,0.064705,-0.031079,0.583382,-0.562638,-0.475557,-0.323972,...,-0.589504,0.373410,-0.232662,-0.053045,-0.141478,1.458140,-0.165308,0.289262,0.212473,-0.045878


In [19]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
# The column that we want to predict.
y_column = ["pts"]

In [20]:
# Splitting the dataset into 'X' and 'Y'column for Training set and Test set
X=nba_normalized[x_columns]
Y=nba_normalized[y_column]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25,random_state=0,shuffle=True)
print("X-TRAINING DATA:\n",X_train)
print("X-TEST DATA:\n",X_test)
print("Y-TRAINING DATA:\n",y_train)
print("Y-TEST DATA:\n",y_test)

X-TRAINING DATA:
           age         g        gs        mp        fg       fga       fg.  \
435  1.784224 -0.602370 -0.862207 -0.942188 -0.878072 -0.855261 -0.521284   
397 -0.121325 -2.024019 -0.862207 -1.376845 -1.122495 -1.150773  0.000000   
159  1.546030  0.266415  1.160825  0.720654  1.350839  1.248571  0.441498   
157 -0.359519  0.700808 -0.862207 -0.211073 -0.534715 -0.432325 -0.561822   
49  -0.597712  0.977240 -0.423883 -0.026065  0.099623 -0.085302  0.988764   
303 -0.835906 -0.325938 -0.626186 -0.573287 -0.697664 -0.700727  0.036116   
52  -0.597712 -0.404919  0.486481  0.199066  0.419702  0.356611  0.411095   
74  -0.597712  0.542847  1.396845  1.508610  0.477898  0.728035 -0.399670   
26  -0.121325  0.779789 -0.525035 -0.318065 -0.534715 -0.679038  1.454954   
276 -0.359519  1.135201  1.902603  1.521984  0.943467  0.595190  1.161052   
4   -0.359519  0.108454  0.149309 -0.319180 -0.331028 -0.475703  1.110379   
282  2.022417  0.898259  1.700300  1.305770  0.861992  0.7

In [21]:
# for k in range(60):
# Create the knn model.
knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto')
# Fit the model on the training data.
knn.fit(X_train, y_train)
# Make predictions on the test set using the fit model.
predictions = knn.predict(X_test)
actual = y_test

#Computing Mean square error and R^2 of the prediction#


mse = (((predictions - actual) ** 2).sum()) / len(predictions)
print("MSE:",mse)
model_score = knn.score(X_train,y_train)
print("coefficient of determination R^2 of the prediction.:",model_score)

MSE: pts    0.034933
dtype: float64
coefficient of determination R^2 of the prediction.: 0.9755850822535296
